## Installing the Dependencies

In [34]:
# !pip install langchain
# !python -m pip install -qU langchain-groq --user
# !pip install "langserve[all]"

In [27]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

As your application becomes more complex, it can be challenging to track and debug the flow of information and LLM invocations. LangSmith is a tool designed to help with this. It allows you to inspect, monitor, and troubleshoot what’s happening inside your LangChain processes, providing better insights into how the chain or agent operates.

The function getpass.getpass() prompts the user to enter the API key while keeping it hidden as they type (it won’t be shown in the terminal or notebook). This ensures that the sensitive information remains secure and is not accidentally exposed in logs or screen captures.

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [14]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

 ········


In [20]:
# Loading the LLM model
model = ChatGroq(model="llama3-8b-8192")
parser = StrOutputParser()

In [17]:
messages = [

    # To set the context for the AI model
    SystemMessage(content="Translate the following from English into Italian"),

    # To represent the user input
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_time': 0.003333333, 'prompt_time': 0.00244165, 'queue_time': 0.01293123, 'total_time': 0.005774983}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-c4f00614-d948-4589-acf8-abfc394d5c84-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28})

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

There is a very common saying that Garbage in Garbage out and is is true for the LLM as well so if we want to get good quality response from the LLM then we will have to provide a good input prompt to the LLM. For doing so we can use the prompt templates which takes the raw user input and build a context around that input.

In [42]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [43]:
# Let's use the LangChain Expression Language (LCEL) to chain together LangChain modules
chain = prompt_template | model | parser

In [44]:
chain.invoke({"language": "Punjabi", "text": "I belive in God for everything"})

'ਮੈਂ ਸਭ ਲਈ ਈਸੁ ਵਿੱਚ ਵਿਸ਼ਵਾਸ ਰੱਖਦਾ ਹਾਂ\n\n(Maen sabh laee Ees vich visvaas rakhandaa haan)\n\nTranslation: I believe in God for everything.'